In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
#from src.scripts.process_tweets import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
from collections import defaultdict
import seaborn as sns
%matplotlib inline

In [3]:
# load a sample of tweets (park tweets, 2 control groups, park names)
day = "0601"
results_dir = Path("../data/processed/combined_tweets/tweet_samples_{}".format(day))
with open(results_dir/Path("park_tweets.json"), 'r') as fp:
    park_tweets = json.load(fp)
with open(results_dir/Path("time_tweets.json"), 'r') as fp:
    time_tweets = json.load(fp)
with open(results_dir/Path("user_tweets.json"), 'r') as fp:
    user_tweets = json.load(fp)
with open(results_dir/Path("park_stops.json"), 'r') as fp:
    park_names = json.load(fp)

# combine all tweets from all cities into lists of tweets
park_tweet_text = []
for city, tweets in park_tweets.items():
    park_tweet_text += tweets
time_tweet_text = []
for city, tweets in time_tweets.items():
    time_tweet_text += tweets
user_tweet_text = []
for city, tweets in user_tweets.items():
      user_tweet_text += tweets
park_stops = []
for city, city_parks in park_names.items():
    park_stops+=city_parks

In [4]:
## load frequent stop words  stop words
stop_file = Path("../src/sentiment/city_stops.json")
with open(stop_file, 'r') as fp:
    stop_dict  = json.load(fp)
    
#load stop words selected from raw wordshifts for probelmatic words
freq_stops = []
for city, words in stop_dict.items():
    freq_stops+=words
freq_stops.append('park')
freq_stops = list(set(freq_stops))

# combine with  names of parks
all_stops = freq_stops + park_stops


# put through ngram filter, lower
all_stops = list(set([x.lower() for x in dict(regex.get_ngrams(' '.join(all_stops), path='../src/func/ngrams.bin')).keys()]))
print(len(all_stops))
# select ones with valence below 4 or greater than 6
stops_w_val=[]
vals_of_stops = {}
words = labmtgen.load_labmt_words("../data/raw/data_labmt_simple.txt")
senti_dict = words.set_index('Word').to_dict()['Happs']
for stop in all_stops:
    if stop in senti_dict.keys():
        val = senti_dict[stop]
        if val <= 4.0 or val >= 6.0:
            #count+=1
            stops_w_val.append(stop)
        vals_of_stops[stop] = val
print(len(stops_w_val))

6973
542


In [13]:
with open("./results/parkstop_vals.json", 'w') as fp:
    json.dump(vals_of_stops,fp)

In [14]:
park_sentis1, control_sentis1 = bootstrap_sentiment(park_tweet_text, time_tweet_text, all_stops,sample=.8,runs=10, bumps=False)

In [87]:
# bootstrap with various controls, stop combos
park_sentis1, control_sentis1 = bootstrap_sentiment(park_tweet_text, control_tweet_text, all_stops,sample=.8,runs=10, bumps=False)
park_sentis2, control_sentis2 = bootstrap_sentiment(park_tweet_text, control_tweet_text, freq_stops,sample=.8,runs=10, bumps=False)
park_sentis3, control_sentis3 = bootstrap_sentiment(park_tweet_text, control_user_text, all_stops,sample=.8,runs=10, bumps=False)
park_sentis4, control_sentis4 = bootstrap_sentiment(park_tweet_text, control_user_text, freq_stops,sample=.8,runs=10, bumps=False)
#time_ctl, user_ctl = bootstrap_sentiment(park_tweet_txt, user_tweet_text, all_stops,sample=.8,runs=10, bumps=False)
#results_dict[city.name]={"park_sentiment":park_sentis,"control_sentiment":control_sentis}#

# save above
results_dict = {"park_sentiment": park_sentis4, "control_sentiment":control_sentis4}
#with open("./results/bootstrap_cities_freqstops_user.json", 'w') as fp:
#    json.dump(results_dict,fp)

In [6]:
# build sampled lists of tweets
# go through city names
park_tweet_samples = []
time_tweet_samples = []
user_tweet_samples = []
#iterate through cities
for city, tweets in park_tweets.items():
    #list of indices to pull from each group
    index_list = range(len(tweets))
    # get random selection integers 
    sample = random.sample(index_list, 1000)
    # add those to each output array
    for i in sample:
        park_tweet_samples.append(park_tweets[city][i])
        time_tweet_samples.append(time_tweets[city][i])
        user_tweet_samples.append(user_tweets[city][i])

In [8]:
# estimate sentiment of sampled version
park_sentis, control_sentis = bootstrap_sentiment(park_tweet_samples, time_tweet_samples, all_stops,sample=.8,runs=10, bumps=False)
#park_sentis_u, control_sentis_u = bootstrap_sentiment(park_tweet_samples, user_tweet_samples, all_stops,sample=.8,runs=10, bumps=False)
#park_sentis_t2, control_sentis_t2 = bootstrap_sentiment(park_tweet_samples, time_tweet_samples, freq_stops,sample=.8,runs=10, bumps=False)
#park_sentis_u2, control_sentis_u2 = bootstrap_sentiment(park_tweet_samples, user_tweet_samples, freq_stops,sample=.8,runs=10, bumps=False)

In [17]:
results_dict = {"park_sentiment": park_sentis1, "control_sentiment":control_sentis1}
with open("./results/bootstrap_allcities_alltweets_allstops_time_0601.json", 'w') as fp:
    json.dump(results_dict,fp)

In [12]:
with open('./results/bootstrap_allcities_allstops_time_0601.json','r') as fp:
    results_dict = json.load(fp)
    
print(np.mean(results_dict['park_sentiment'])-np.mean(results_dict['control_sentiment']))
sentis = []
for i in range(10):
    sentis.append(results_dict['park_sentiment'][i]-results_dict['control_sentiment'][i])
print(np.min(sentis),np.mean(sentis),np.max(sentis))

0.09205348420336268
0.0851745898883971 0.09205348420336223 0.09812511365308296
